# **워드 임베딩 + 클러스터링**

In [ ]:
# 기본 패키지 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 데이터 파일 불러오기
df_1 = pd.read_csv('/content/book.csv', encoding='euc-kr')

# df_1.head()

In [ ]:
df_1.tail()

In [ ]:
# 전처리 - price  object => 수치형
# df['Price'] = pd.to_numeric(df['Price'].str.replace('[^\d.]', ''), errors='coerce')

# 'Pdate' 컬럼을 날짜 타입으로 변환 (한국어 날짜 형식에 맞춤)
# df['Pdate'] = pd.to_datetime(df['Pdate'], format='%Y년 %m월 %d일', errors='coerce')


In [ ]:
# 전처리 - 공백 제거
df_1 = df_1.dropna(subset=['Title'])
df_1 = df_1[df_1['Title'].str.strip() != '']

# df_2 = df_1.dropna(subset=['Publisher'])
# df_2 = df_2[df_2['Publisher'].str.strip() != '']

In [ ]:
# 전처리 - 언어별 구분
# df = df_2[df_1['Title'].str.contains('[A-Za-z]', regex=True)]
# 한국어 [가-힣]
# 영어[A-Za-z]
# 일본어 [ぁ-んァ-ン]


In [ ]:
# 전처리 - 언어별 구분
import re

# 영어 제목만
def is_english_title(text):
    # 영어 알파벳, 공백, 일부 특수문자(예: ',!?.)만 허용
    return bool(re.match(r'^[a-zA-Z0-9 .,\-\'!?]+$', text))

# 영어로만 구성된 책 제목만 가져옴
df_e = df_1[df_1['Title'].apply(is_english_title)]

# 한국 제목만
def is_korean_title(text):
    # 영어 알파벳, 공백, 일부 특수문자(예: ',!?.)만 허용
    return bool(re.match(r'^[가-힣0-9 .,\-\'!?]+$', text))

# 한국어로만 구성된 책 제목만 가져옴
df_k = df_1[df_1['Title'].apply(is_korean_title)]


## df_k,   df_e  변수명 변경 시... 관련된 부분 모두 수정해 주어야 함

In [ ]:
# 벡터화 + 클러스터링을 위한 패키지
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


**Tf-idf** <br>
단어의 출현 빈도 및 상대 빈도 활용

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=100) # 전체 단어 집합에서 TF-IDF 값이 가장 높은 상위 n개의 단어만을 선택하여 특성 벡터를 생성
X_tfidf = tfidf_vectorizer.fit_transform(df_k)

In [ ]:
# 클러스터링 수행
kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(X_tfidf)

### 이후 아래 #123 코드 셀로 이동하여 코드 실행

KMeans(n_clusters=10, random_state=42)

**Word2Vec** <br>
주변단어들을 학습에 사용 <br> <br>
CBOW<br>
주변 단어들(문맥)-타겟 단어의 앞뒤에 위치한 단어들-을 기반으로 타겟 단어 예측<br>
ex "The cat sits on the ___" 빈칸에 들어갈 단어(타겟 단어) 예측<br><br>
Skip-gram<br>
특정 단어를 입력으로 받아, 그 단어 주변의 문맥 단어를 예측<br>
ex "cat"이라는 단어가 주어졌을 때, 이 단어 주변에 위치할 가능성이 높은 단어("The", "sits", "on") 예측

In [ ]:
################# Word2Vec  #################
from gensim.models import Word2Vec
import pandas as pd
import numpy as np

# 책 제목을 단어 리스트로 변환
sentences = [title.split() for title in df_k['Title']]

# Word2Vec 모델 학습
word2vec_model = Word2Vec(sentences, vector_size=64, window=5, min_count=1)
   # 책 제목에서 각 단어의 벡터 표현 학습
   # 학습을 통해 책 제목을 구성하는 단어들 간의 관계와 문맥을 바탕으로 각 단어의 의미를 반영하는 벡터 생성


In [ ]:
# 책 제목 -> 벡터값 조회 및 제목단위 평균값 산출
# 학습된 word2vec 모델을 사용하여 각 책 제목을 구성하는 단어들의 벡터 값 조회
def vectorize_w_word2vec(text):
    words = text.split()
    word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if len(word_vectors) == 0:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(word_vectors, axis=0)

# 각 책 제목을 벡터화
title_vectors_k = np.array([vectorize_w_word2vec(title) for title in df_k['Title']])

### 이 후 아래 클러스터링 코드 실행하여 결과 확인

**FastText** <br>
주변 단어들을 학습에 사용 =  word2vec <br>
서브 워드 사용 <br>

In [ ]:
################# FastText  #################
from gensim.models import FastText
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

# FastText 모델 학습 (로컬 데이터)
# sentences = [title.split() for title in df_e['Title']]
sentences = [title.split() for title in df_k['Title']]
fasttext_model = FastText(sentences, vector_size=64, window=5, min_count=1)
   # 책 제목에서 각 단어와 서브워드(subword)의 벡터 표현 학습
   # 학습을 통해 책 제목을 구성하는 단어들 간의 관계와 문맥을 바탕으로 각 단어의 의미를 반영하는 벡터 생성



In [ ]:
# 책 제목 -> 벡터값 조회 및 제목단위 평균값 산출
# 학습된 FastText 모델을 사용하여 각 책 제목을 구성하는 단어들의 벡터 값 조회
def vectorize_w_fasttext(text):
    words = text.split()
    word_vectors = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
    if len(word_vectors) == 0:
        return np.zeros(fasttext_model.vector_size)
    return np.mean(word_vectors, axis=0)

# 제목 벡터화
title_vectors_k = np.array([vectorize_w_fasttext(title) for title in df_k['Title']])

**아래 클러스터링 과정은 동일**

In [ ]:
# K-평균 클러스터링 수행
kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(title_vectors_k)

# 클러스터 할당 결과
df_k['Cluster'] = kmeans.labels_

In [ ]:
# 123 코드 셀
# 클러스터 별로 데이터 확인
for cluster in range(10): # 클러스터의 수에 따라 범위 조정
    print(f"Cluster {cluster}:")
    print(df_k[df_k['Cluster'] == cluster]['Title'].head(), '\n') # 각 클러스터에 속한 책 제목 출력


Cluster 0:
409     묻고 답하는 한국사카페 1
442         두근두근 중국어 1
476          런웨이의 연인 1
518      우리에게는 벽이 있다 1
520    반쪽 달이 떠오르는 하늘 1
Name: Title, dtype: object 

Cluster 1:
1719     쿠키런 서바이벌 대작전 7
1738     쿠키런 서바이벌 대작전 2
1739     쿠키런 서바이벌 대작전 1
1740     쿠키런 서바이벌 대작전 3
1768    쿠키런 서바이벌 대작전 16
Name: Title, dtype: object 

Cluster 2:
25         이화림 회고록
126         경영의 명의
191         척추영상진단
192    건축견적이야기 세트 
262    가족신탁 이론과 실무
Name: Title, dtype: object 

Cluster 3:
98                         창의적 공학설계
165              4차 산업혁명 시대의 운영관리혁신
173                       알기쉬운 해부생리
277    유패스 지텔프 최신 기출유형 공식 기본서 문법 독해
342                       프랑스 엄마 수업
Name: Title, dtype: object 

Cluster 4:
1851    수학도둑 39
1867    수학도둑 50
1868    수학도둑 41
1904    수학도둑 36
1912    수학도둑 42
Name: Title, dtype: object 

Cluster 5:
218    주류 창업을 위한 주류면허 길라잡이
344        생각하는 아이 기다리는 엄마
345     우리는 더 많은 민주주의를 원한다
350                 공부 추진력
351       맛있는 햄버거의 무서운 이야기
Name: Title, dtype: object 

Cluster 6:
1711         쿠키런 한자런 7
1

In [ ]:
similar_words = fasttext_model.wv.most_similar('설민석 ')

for word, similarity in similar_words:
    print(f"Word: {word}, similarity : {similarity}")

Word: 설민석의, similarity : 0.7448292970657349
Word: 대모험, similarity : 0.5696137547492981
Word: 편지, similarity : 0.49376335740089417
Word: 한국사회와, similarity : 0.48239025473594666
Word: 혼자다, similarity : 0.4770514965057373
Word: 한국사2, similarity : 0.4603557884693146
Word: 놀이한국사, similarity : 0.45072123408317566
Word: 중국사, similarity : 0.4335116744041443
Word: 피트니스, similarity : 0.4328833222389221
Word: 한국사회사연구, similarity : 0.4320783019065857


In [ ]:
# 유사한 단어
fasttext_model.wv.most_similar('신사')

[('언어가', 0.446243554353714),
 ('독성학', 0.44315633177757263),
 ('시인은', 0.44257327914237976),
 ('공작부인', 0.4249315857887268),
 ('프랑스', 0.4107048809528351),
 ('공략', 0.4074508547782898),
 ('배워', 0.4061374068260193),
 ('활용률', 0.3938716650009155),
 ('미련한', 0.39299649000167847),
 ('글방', 0.3900793194770813)]